In [876]:
import itertools

In [877]:
dataset = [
    ['A', 'B'],
    ['A', 'C'],
    ['A', 'D'],
    ['B', 'C', 'D'],
    ['A', 'B', 'C', 'D'],
]

In [878]:
def generate_frequent_1_itemsets(dataset, min_support):
    frequent_1_itemsets = {}
    for transaction in dataset:
        for item in transaction:
            if item in frequent_1_itemsets:
                frequent_1_itemsets[item] += 1
            else:
                frequent_1_itemsets[item] = 1

    frequent_1_itemsets = {item: support for item, support in frequent_1_itemsets.items() if support >= min_support}
    return frequent_1_itemsets

In [879]:
def generer_combinaisons(liste):
    combinaisons = []
    if(len(liste)>0):
        for r in range(len(liste)-1 , len(liste)):
            combinaisons.extend(list(itertools.combinations(liste, r)))
    combinaisons_uniques = set(combinaisons)
    return combinaisons_uniques

In [880]:
def calculate_support(dataset, itemsets):
    itemset_counts = {}
    for transaction in dataset:
        for itemset in itemsets:
            if set(itemset).issubset(transaction):
                if tuple(itemset) in itemset_counts:
                    itemset_counts[tuple(itemset)] += 1
                else:
                    itemset_counts[tuple(itemset)] = 1
    return {itemset: support for itemset, support in itemset_counts.items()}



In [881]:
def recu(current_itemset=None,C1={},C2={},dataset=None,min_support=0,frequent_max={}):
    global generation
    global updated_itemset
    itemset_counts ={}
    if current_itemset ==set():
        print("condition d'arret")
        print("les itemset frequent max",frequent_max)
        return frequent_max   
    else:
        i=1
        for itemset in current_itemset:
            generation = generer_combinaisons(list(itemset))
            print("le itemset numéro",i," de longeur ",len(list(itemset)))
            i=i+1
            print(generation)
            itemset_counts = calculate_support(dataset, generation)
            for cle, valeur in itemset_counts.items():
                if valeur>=min_support:
                    frequent_max[cle] = valeur
                    generation.remove(cle)
            print(itemset_counts)
            print(generation)
            current_itemset= generation.copy()
            print("valeur vide",current_itemset)
            if not set(generation) in C1:
                C1.update(generation)
                print('coucou C1',C1)
        return recu(current_itemset,C1,C2,dataset,min_support,frequent_max)

In [882]:
def support_counts(itemset, data):
    support_count = 0
    for transaction in data:
        if itemset.issubset(transaction):
            support_count += 1

    
    support = support_count / len(data)

    return support

In [883]:
def get_all_subsets(itemset):
    # Cette fonction génère tous les sous-ensembles non vides d'un ensemble donné.
    subsets = []
    for i in range(1, len(itemset)):
        subsets.extend(itertools.combinations(itemset, i))
    return subsets

In [884]:
def generate_rules(frequent_itemsets, min_confidence, dataset=None):
    # Cette fonction génère les règles d'association à partir des ensembles fréquents.
    rules = []
    for cle, valeur in frequent_itemsets.items():
        #if len(itemset) > 1:
            subsets = [frozenset(subset) for subset in get_all_subsets(cle)]
            for subset in subsets:
                remaining = set(cle).difference(subset)
                confidence = valeur / support_counts(set(subset), dataset)
                if confidence >= min_confidence:
                    rules.append((subset, remaining, confidence))
    return rules

In [885]:
def apriori(dataset, min_support=0, minconfidence=0):
    frequent_itemsets = {}
    frequent_max={}
    k = 1
    frequent_1_itemsets = generate_frequent_1_itemsets(dataset, min_support)
    frequent_itemsets.update(frequent_1_itemsets)
    sorted_frequent_itemsets = dict(sorted(frequent_itemsets.items(), key=lambda item:item[1], reverse=True))
    print('les itemsets frequents ordre décroissant',sorted_frequent_itemsets)
    print("support de 1-items ",sorted_frequent_itemsets)
    resultat = generer_combinaisons(list(sorted_frequent_itemsets.keys()))
    print("Génération des premiers subsets",resultat)
    for x in resultat:
        itemset_counts = calculate_support(dataset, resultat)
    print("support de 1-items",itemset_counts)
    for cle, valeur in itemset_counts.items():
            if valeur>=min_support:
                frequent_max[cle] = valeur
                resultat.remove(cle)    
    print("les itemsets non frequent dont on va déploier",resultat)
    c=set()
    c2=set()
    r=recu(resultat,c,c2,dataset,min_support,frequent_max)
    print ("voici le resulltat",r)
    print(type(r))
    rules = generate_rules(frequent_max, minconfidence, dataset)

    # Affichage des règles d'association
    for antecedent, consequent, confidence in rules:
        print(f"{antecedent} => {consequent} (Confidence: {confidence})")

In [886]:
#le premier paramétre est celui de la dataset , le second le minimum support , le troisieme est le minimum de confience
apriori(dataset,2,3)

les itemsets frequents ordre décroissant {'A': 4, 'C': 4, 'D': 3, 'B': 2}
support de 1-items  {'A': 4, 'C': 4, 'D': 3, 'B': 2}
Génération des premiers subsets {('A', 'C', 'D'), ('C', 'D', 'B'), ('A', 'C', 'B'), ('A', 'D', 'B')}
support de 1-items {('C', 'D', 'B'): 2, ('A', 'C', 'D'): 1, ('A', 'C', 'B'): 1, ('A', 'D', 'B'): 1}
les itemsets non frequent dont on va déploier {('A', 'C', 'D'), ('A', 'C', 'B'), ('A', 'D', 'B')}
le itemset numéro 1  de longeur  3
{('C', 'D'), ('A', 'C'), ('A', 'D')}
{('A', 'C'): 3, ('A', 'D'): 2, ('C', 'D'): 2}
set()
valeur vide set()
coucou C1 set()
le itemset numéro 2  de longeur  3
{('A', 'C'), ('C', 'B'), ('A', 'B')}
{('A', 'C'): 3, ('C', 'B'): 2, ('A', 'B'): 1}
{('A', 'B')}
valeur vide {('A', 'B')}
coucou C1 {('A', 'B')}
le itemset numéro 3  de longeur  3
{('D', 'B'), ('A', 'D'), ('A', 'B')}
{('A', 'D'): 2, ('D', 'B'): 2, ('A', 'B'): 1}
{('A', 'B')}
valeur vide {('A', 'B')}
coucou C1 {('A', 'B')}
le itemset numéro 1  de longeur  2
{('A',), ('B',)}
{('A',